Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD       Naive SE       MCSE      ESS
alpha 154.5940922 0.26507033 0.0041911299 0.0038050199 1000
 beta   0.9051552 0.04229885 0.0006688036 0.0006707288 1000
sigma   5.1022487 0.19402978 0.0030678802 0.0027726686 1000

Quantiles:
          2.5%        25.0%       50.0%      75.0%        97.5%   
alpha 154.0739750 154.41300000 154.595000 154.7742500 155.10605000
 beta   0.8223087   0.87716125   0.904595   0.9334748   0.98891855
sigma   4.7291695   4.96768750   5.099400   5.2255875   5.50065950



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.566 0.858592 4.81801; 154.617 0.866835 4.84578; … ; 154.774 0.896076 5.2577; 154.292 0.941172 5.17338]

Union{Missing, Float64}[154.735 0.888608 5.21057; 154.295 0.871426 5.13241; … ; 154.782 0.895919 5.1421; 154.768 0.96443 5.18754]

Union{Missing, Float64}[154.551 0.944522 5.12955; 154.504 0.884066 5.12139; … ; 154.725 0.97667 5.32538; 154.425 0.854561 4.93327]

Union{Missing, Float64}[154.529 0.894845 5.0997; 154.649 0.911117 5.05649; … ; 154.911 0.874642 5.22804; 154.91 0.894703 5.14682]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD       Naive SE       MCSE      ESS
alpha 154.5940922 0.26507033 0.0041911299 0.0038050199 1000
 beta   0.9051552 0.04229885 0.0006688036 0.0006707288 1000
sigma   5.1022487 0.19402978 0.0030678802 0.0027726686 1000

Quantiles:
          2.5%        25.0%       50.0%      75.0%        97.5%   
alpha 154.0739750 154.41300000 154.595000 154.7742500 155.10605000
 beta   0.8223087   0.87716125   0.904595   0.9334748   0.98891855
sigma   4.7291695   4.96768750   5.099400   5.2255875   5.50065950



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*